# Web Crawler 

## 實作1: 爬取PTT八卦版的文章資訊

### 以PTT八卦版為例 : https://www.ptt.cc/bbs/Gossiping/index.html

### Step1. import 套件

In [1]:
import requests
from bs4 import BeautifulSoup

# 把Warning訊息去掉
requests.packages.urllib3.disable_warnings()

### Step2. 將網頁Get下來

In [2]:
# 設定proxy 在公司內需使用
http_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
https_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
ftp_proxy = "http://auhqproxy.corpnet.auo.com:8080/"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }
            
#r = requests.get(url, proxies=proxyDict, verify=False)

In [3]:
# 要爬的網站
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

### Step3. 處理PTT網站詢問是否已滿18歲的問題
1. 進入網站後會看到讓使用者點選「是否已滿18歲」按鈕

2. 可開啟瀏覽器開發者模式(F12)，並點選至network(網路)，觀察點選「已滿18歲」後，會送給伺服器之封包內容。

3. 發現封包以POST發送，並且封包內容有cookie和參數

4. 我們的程式碼可以透過POST的方式，送出cookie和特定的參數

5. 可以藉由requests的一個參數來儲存自身的cookie
   Session會將你送出的requests所收到的cookies全部儲存起來並且在發送下一次請求時送出對應的參數。

In [4]:
# 處理是否已滿18歲的問題
payload = {
    "from":"/bbs/Gossiping/index.html",
    "yes":"yes"
}

rs = requests.session()

# 送出資訊已滿18歲的資訊
# verify=False 是為了避免SSL認證
res = rs.post("https://www.ptt.cc/ask/over18", proxies=proxyDict, verify=False, data=payload)

# 將此頁面的HTML GET下來
res = rs.get(url, proxies=proxyDict, verify=False)

# 印出HTML
print(res.text)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/co

### Step4. 擷取各文章標題
```html=
<div class="title">	
    <a href="/bbs/Gossiping/M.1650337301.A.456.html">[新聞] 基隆無症狀、輕症 居家照護今啟動</a>
</div>
```

In [5]:
# 將網頁資料以html.parser
soup = BeautifulSoup(res.text, "html.parser")

# 取HTML標中的 <div class="title"></div> 中的<a>標籤存入title
title = soup.select("div.title a")
print(title)

[<a href="/bbs/Gossiping/M.1650357080.A.422.html">[問卦] 馬斯克買下Twitter有什麼好處？</a>, <a href="/bbs/Gossiping/M.1650357130.A.DEF.html">Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：</a>, <a href="/bbs/Gossiping/M.1650357130.A.F72.html">Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：</a>, <a href="/bbs/Gossiping/M.1650357154.A.BFD.html">[問卦] 女:一百歲的時候我很老耶 男:我也很老阿</a>, <a href="/bbs/Gossiping/M.1650357197.A.423.html">[問卦] 要怎麼解釋，“請大家自主應變”？</a>, <a href="/bbs/Gossiping/M.1650357208.A.31C.html">Re: [問卦] 國共內戰三大會戰都打輸 太爛了吧？</a>, <a href="/bbs/Gossiping/M.1650357300.A.2A1.html">Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：</a>, <a href="/bbs/Gossiping/M.1650357317.A.EDB.html">[問卦] =.= 叫你吃兩個便當的好還是自己想辦法</a>, <a href="/bbs/Gossiping/M.1650357353.A.643.html">[問卦] 「穿條紋衣的男孩」是驚悚片吧！？</a>, <a href="/bbs/Gossiping/M.1650357376.A.DE4.html">[新聞] 上海染疫死亡人數引質疑 網民批評聲浪四起</a>, <a href="/bbs/Gossiping/M.1650357421.A.074.html">[問卦] 世界落後台灣多少年了</a>, <a href="/bbs/Gossiping/M.1650357426.A.2A3.html">Re: [問卦] 我們比其他國家晚了一年爆發為何要民眾</a>, <a href="/bbs/Gossiping/M.1650357466.A.902.html

In [7]:
# 將爬下來的文章標題印出來
for s in title:
    # https://www.ptt.cc/
    print("https://www.ptt.cc/" + s["href"], s.text)

https://www.ptt.cc//bbs/Gossiping/M.1650357080.A.422.html [問卦] 馬斯克買下Twitter有什麼好處？
https://www.ptt.cc//bbs/Gossiping/M.1650357130.A.DEF.html Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：
https://www.ptt.cc//bbs/Gossiping/M.1650357130.A.F72.html Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：
https://www.ptt.cc//bbs/Gossiping/M.1650357154.A.BFD.html [問卦] 女:一百歲的時候我很老耶 男:我也很老阿
https://www.ptt.cc//bbs/Gossiping/M.1650357197.A.423.html [問卦] 要怎麼解釋，“請大家自主應變”？
https://www.ptt.cc//bbs/Gossiping/M.1650357208.A.31C.html Re: [問卦] 國共內戰三大會戰都打輸 太爛了吧？
https://www.ptt.cc//bbs/Gossiping/M.1650357300.A.2A1.html Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：
https://www.ptt.cc//bbs/Gossiping/M.1650357317.A.EDB.html [問卦] =.= 叫你吃兩個便當的好還是自己想辦法
https://www.ptt.cc//bbs/Gossiping/M.1650357353.A.643.html [問卦] 「穿條紋衣的男孩」是驚悚片吧！？
https://www.ptt.cc//bbs/Gossiping/M.1650357376.A.DE4.html [新聞] 上海染疫死亡人數引質疑 網民批評聲浪四起
https://www.ptt.cc//bbs/Gossiping/M.1650357421.A.074.html [問卦] 世界落後台灣多少年了
https://www.ptt.cc//bbs/Gossiping/M.1650357426.A.2A3.html Re: [問卦] 我們比其他國家晚了一年爆發為何要民眾
